<a href="https://colab.research.google.com/github/Tonry12/Data_joyx2/blob/main/Lottery_Prediction_model_LSTM_(-1)(100)(0.01).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PrepareDATA

Link: https://medium.com/@polanitzer/forecasting-the-next-winning-numbers-in-the-texas-lottery-mega-millions-drawing-using-a-deep-4c37ee6d30f0

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os
import pickle

In [3]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
lotto = pd.read_csv('/content/drive/MyDrive/SPN/Predictive lottery 30.csv')

In [5]:
lotto = lotto.drop(columns=['Date','Unnamed: 1','Unnamed: 2','Unnamed: 3'])

In [6]:
lotto

,DATE,รางวัลที่ 1,2 ตัวบน,3 ตัวบน,2 ตัวล่าง,3 ตัวหน้า,3 ตัวหน้า.1,3 ตัวล่าง,3 ตัวล่าง.1,weekday_dum,weekday
0,6/1/2024,530593,93,593,42,194,364,734,421,7,Saturday
1,5/16/2024,205690,90,690,60,747,885,70,137,5,Thursday
2,5/2/2024,980116,16,116,17,104,763,634,833,4,Wednesday
3,4/16/2024,943598,98,598,79,727,729,154,200,3,Tuesday
4,4/1/2024,803481,81,481,90,122,809,559,947,2,Monday
...,...,...,...,...,...,...,...,...,...,...,...
718,5/16/1994,2071764,64,764,29,93,544,535,746,2,Monday
719,5/2/1994,3072877,77,877,69,535,317,170,521,2,Monday
720,4/16/1994,7893913,13,913,21,785,982,151,594,7,Saturday
721,4/1/1994,4269815,15,815,29,675,35,704,868,6,Friday


In [7]:
lotto.rename(columns={'DATE': 'Date','รางวัลที่ 1': '1st','3 ตัวบน':'top3','2 ตัวล่าง':'up2'},inplace=True)

In [8]:
lotto

,Date,1st,2 ตัวบน,top3,up2,3 ตัวหน้า,3 ตัวหน้า.1,3 ตัวล่าง,3 ตัวล่าง.1,weekday_dum,weekday
0,6/1/2024,530593,93,593,42,194,364,734,421,7,Saturday
1,5/16/2024,205690,90,690,60,747,885,70,137,5,Thursday
2,5/2/2024,980116,16,116,17,104,763,634,833,4,Wednesday
3,4/16/2024,943598,98,598,79,727,729,154,200,3,Tuesday
4,4/1/2024,803481,81,481,90,122,809,559,947,2,Monday
...,...,...,...,...,...,...,...,...,...,...,...
718,5/16/1994,2071764,64,764,29,93,544,535,746,2,Monday
719,5/2/1994,3072877,77,877,69,535,317,170,521,2,Monday
720,4/16/1994,7893913,13,913,21,785,982,151,594,7,Saturday
721,4/1/1994,4269815,15,815,29,675,35,704,868,6,Friday


In [9]:
ls= lotto.iloc[:,9:11]

In [10]:
lu= lotto.iloc[:,0:5]

In [11]:
lt = pd.concat([lu, ls], axis=1)

In [12]:
lt

,Date,1st,2 ตัวบน,top3,up2,weekday_dum,weekday
0,6/1/2024,530593,93,593,42,7,Saturday
1,5/16/2024,205690,90,690,60,5,Thursday
2,5/2/2024,980116,16,116,17,4,Wednesday
3,4/16/2024,943598,98,598,79,3,Tuesday
4,4/1/2024,803481,81,481,90,2,Monday
...,...,...,...,...,...,...,...
718,5/16/1994,2071764,64,764,29,2,Monday
719,5/2/1994,3072877,77,877,69,2,Monday
720,4/16/1994,7893913,13,913,21,7,Saturday
721,4/1/1994,4269815,15,815,29,6,Friday


In [13]:
lt = lt.drop(columns=['2 ตัวบน'])

In [14]:
more = lt[lt['1st'] >= 999999]
more.count()

Date           20
1st            20
top3           20
up2            20
weekday_dum    20
weekday        20
dtype: int64

In [15]:
lt = lt[lt['1st'] <= 999999]

In [16]:
lt['up2'] = lt['up2'].apply(lambda x: '{:02}'.format(x))

In [17]:
lt['top3'] = lt['top3'].apply(lambda x: '{:03}'.format(x))

In [18]:
lt['Date'] = pd.to_datetime(lt['Date'])

In [19]:
lt['Year'] = pd.to_datetime(lt['Date']).dt.year
lt['Month'] = pd.to_datetime(lt['Date']).dt.month
lt['Day'] = pd.to_datetime(lt['Date']).dt.day

In [20]:
lt_1 = lt.copy()

In [21]:
lt_1['up2'] = lt_1['up2'].astype(str)

# Split each value in the 'digi' column into its individual digits
lt_1['Tens digit'] = lt_1['up2'].apply(lambda x: int(x[0]))
lt_1['Units digit'] = lt_1['up2'].apply(lambda x: int(x[1]) if len(x) > 1 else 0)

In [22]:
lt_1['top3'] = lt_1['top3'].astype(str)

# Split each value in the 'digi' column into its individual digits
lt_1['c3_r1'] = lt_1['top3'].apply(lambda x: int(x[0]))
lt_1['c3_r2'] = lt_1['top3'].apply(lambda x: int(x[1]))
lt_1['c3_r3'] = lt_1['top3'].apply(lambda x: int(x[2]))

In [23]:
lt_2 = lt_1.copy()

# LSTM Model

In [24]:
lt_2

,Date,1st,top3,up2,weekday_dum,weekday,Year,Month,Day,Tens digit,Units digit,c3_r1,c3_r2,c3_r3
0,2024-06-01,530593,593,42,7,Saturday,2024,6,1,4,2,5,9,3
1,2024-05-16,205690,690,60,5,Thursday,2024,5,16,6,0,6,9,0
2,2024-05-02,980116,116,17,4,Wednesday,2024,5,2,1,7,1,1,6
3,2024-04-16,943598,598,79,3,Tuesday,2024,4,16,7,9,5,9,8
4,2024-04-01,803481,481,90,2,Monday,2024,4,1,9,0,4,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,1995-03-16,646847,847,67,5,Thursday,1995,3,16,6,7,8,4,7
699,1995-03-01,411454,454,71,4,Wednesday,1995,3,1,7,1,4,5,4
700,1995-02-16,869070,070,95,5,Thursday,1995,2,16,9,5,0,7,0
701,1995-02-01,198162,162,48,4,Wednesday,1995,2,1,4,8,1,6,2


In [25]:
sh = lt_2.shape

In [26]:
z = 1

In [27]:
lt_3 = lt_2.iloc[z:sh[0], 0:sh[1]]

In [28]:
lt_3

,Date,1st,top3,up2,weekday_dum,weekday,Year,Month,Day,Tens digit,Units digit,c3_r1,c3_r2,c3_r3
1,2024-05-16,205690,690,60,5,Thursday,2024,5,16,6,0,6,9,0
2,2024-05-02,980116,116,17,4,Wednesday,2024,5,2,1,7,1,1,6
3,2024-04-16,943598,598,79,3,Tuesday,2024,4,16,7,9,5,9,8
4,2024-04-01,803481,481,90,2,Monday,2024,4,1,9,0,4,8,1
5,2024-03-16,997626,626,78,7,Saturday,2024,3,16,7,8,6,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,1995-03-16,646847,847,67,5,Thursday,1995,3,16,6,7,8,4,7
699,1995-03-01,411454,454,71,4,Wednesday,1995,3,1,7,1,4,5,4
700,1995-02-16,869070,070,95,5,Thursday,1995,2,16,9,5,0,7,0
701,1995-02-01,198162,162,48,4,Wednesday,1995,2,1,4,8,1,6,2


## ไม่ล่วงหน้า

In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [30]:
dfp = lt_3.copy()

In [31]:
df = dfp.reset_index()

In [32]:
df

,index,Date,1st,top3,up2,weekday_dum,weekday,Year,Month,Day,Tens digit,Units digit,c3_r1,c3_r2,c3_r3
0,1,2024-05-16,205690,690,60,5,Thursday,2024,5,16,6,0,6,9,0
1,2,2024-05-02,980116,116,17,4,Wednesday,2024,5,2,1,7,1,1,6
2,3,2024-04-16,943598,598,79,3,Tuesday,2024,4,16,7,9,5,9,8
3,4,2024-04-01,803481,481,90,2,Monday,2024,4,1,9,0,4,8,1
4,5,2024-03-16,997626,626,78,7,Saturday,2024,3,16,7,8,6,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,698,1995-03-16,646847,847,67,5,Thursday,1995,3,16,6,7,8,4,7
698,699,1995-03-01,411454,454,71,4,Wednesday,1995,3,1,7,1,4,5,4
699,700,1995-02-16,869070,070,95,5,Thursday,1995,2,16,9,5,0,7,0
700,701,1995-02-01,198162,162,48,4,Wednesday,1995,2,1,4,8,1,6,2


In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [34]:
df2 = df.copy()

In [35]:
df1 = df.copy()
df.drop(['index','Date','1st','weekday_dum','weekday','Year','Month','Day','Tens digit','Units digit','c3_r1','c3_r2','c3_r3'], axis=1, inplace=True)

number_of_features = df.shape[1]

In [36]:
df

,top3,up2
0,690,60
1,116,17
2,598,79
3,481,90
4,626,78
...,...,...
697,847,67
698,454,71
699,070,95
700,162,48


In [37]:
df1.drop(['index','1st','weekday_dum','weekday','Year','Month','Day','Tens digit','Units digit','c3_r1','c3_r2','c3_r3'], axis=1, inplace=True)

In [38]:
df1

,Date,top3,up2
0,2024-05-16,690,60
1,2024-05-02,116,17
2,2024-04-16,598,79
3,2024-04-01,481,90
4,2024-03-16,626,78
...,...,...,...
697,1995-03-16,847,67
698,1995-03-01,454,71
699,1995-02-16,070,95
700,1995-02-01,162,48


In [39]:
window_length = 7
window_length

7

In [40]:
train = df.copy()
train.head((window_length+1))

,top3,up2
0,690,60
1,116,17
2,598,79
3,481,90
4,626,78
5,603,79
6,395,43
7,063,09


In [41]:
train_rows = train.values.shape[0]
train_samples = np.empty([ train_rows - window_length, window_length, number_of_features], dtype=float)
train_labels = np.empty([ train_rows - window_length, number_of_features], dtype=float)
for i in range(0, train_rows-window_length):
    train_samples[i] = train.iloc[i : i+window_length, 0 : number_of_features]
    train_labels[i] = train.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [42]:
train.values.shape[0]

702

In [43]:
train_samples[0]

array([[690.,  60.],
       [116.,  17.],
       [598.,  79.],
       [481.,  90.],
       [626.,  78.],
       [603.,  79.],
       [395.,  43.]])

In [44]:
train_labels[0]

array([63.,  9.])

In [45]:
scaler = StandardScaler()
transformed_dataset = scaler.fit_transform(train.values)
scaled_train_samples = pd.DataFrame(data=transformed_dataset, index=train.index)

In [46]:
scaled_train_samples.head(window_length+1)

,0,1
0,0.646511,0.279825
1,-1.380838,-1.210434
2,0.321570,0.938311
3,-0.091670,1.319540
4,0.420465,0.903654
5,0.339230,0.938311
6,-0.395419,-0.309348
7,-1.568032,-1.487692


In [47]:
x_train = np.empty([ train_rows - window_length, window_length, number_of_features], dtype=float)
y_train = np.empty([ train_rows - window_length, number_of_features], dtype=float)

for i in range(0, train_rows-window_length):
    x_train[i] = scaled_train_samples.iloc[i : i+window_length, 0 : number_of_features]
    y_train[i] = scaled_train_samples.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [48]:
y_train

array([[-1.56803198, -1.48769192],
       [ 1.66724904,  0.3144819 ],
       [ 0.13084375,  1.28488319],
       ...,
       [-1.54330822,  1.49282632],
       [-1.21836733, -0.13606156],
       [-0.42014297, -0.41331907]])

In [49]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mse

In [50]:
# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a third Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))


In [51]:
model.compile(optimizer=Adam(learning_rate=0.005), loss ='mse', metrics=['accuracy'])

In [52]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, verbose=2)

Epoch 1/100
6/6 - 21s - loss: 1.1322 - accuracy: 0.4863 - 21s/epoch - 4s/step
Epoch 2/100
6/6 - 5s - loss: 1.0433 - accuracy: 0.4820 - 5s/epoch - 752ms/step
Epoch 3/100
6/6 - 4s - loss: 1.0241 - accuracy: 0.5180 - 4s/epoch - 615ms/step
Epoch 4/100
6/6 - 3s - loss: 1.0238 - accuracy: 0.5050 - 3s/epoch - 512ms/step
Epoch 5/100
6/6 - 3s - loss: 1.0028 - accuracy: 0.4849 - 3s/epoch - 505ms/step
Epoch 6/100
6/6 - 4s - loss: 1.0093 - accuracy: 0.4892 - 4s/epoch - 636ms/step
Epoch 7/100
6/6 - 4s - loss: 1.0008 - accuracy: 0.4906 - 4s/epoch - 740ms/step
Epoch 8/100
6/6 - 3s - loss: 1.0049 - accuracy: 0.4849 - 3s/epoch - 503ms/step
Epoch 9/100
6/6 - 3s - loss: 1.0057 - accuracy: 0.4863 - 3s/epoch - 575ms/step
Epoch 10/100
6/6 - 3s - loss: 0.9984 - accuracy: 0.4849 - 3s/epoch - 567ms/step
Epoch 11/100
6/6 - 5s - loss: 0.9971 - accuracy: 0.5540 - 5s/epoch - 809ms/step
Epoch 12/100
6/6 - 3s - loss: 0.9930 - accuracy: 0.4993 - 3s/epoch - 508ms/step
Epoch 13/100
6/6 - 3s - loss: 0.9930 - accuracy: 0

### Prediction without rounding up or down the results

In [53]:
print('-' * 40)
print('Prediction vs. GoundTruth without rounding up or down')
for i in range(1,10):
  test = df1.copy()
  test = test.tail((window_length+10-i))
  test = test.head((window_length+1))
  test_Date = df1.iloc[ (test.tail().index[-1]) ]['Date']
  test1 = test.head((window_length))
  test1.drop(['Date'], axis=1, inplace=True)
  test1 = np.array(test1)
  x_test = scaler.transform(test1)
  y_test_pred = model.predict(np.array([x_test]))
  y_test_true = test.drop(['Date'], axis=1, inplace=True)
  y_test_true = test.tail(1)
  print('Drawing  Date', test_Date)
  print('Prediction:\t', scaler.inverse_transform(y_test_pred).astype(int)[0])
  print('GoundTruth:\t', np.array(y_test_true)[0])
  print('-' * 40)

----------------------------------------
Prediction vs. GoundTruth without rounding up or down
1/1 [==============================] - 3s 3s/step
Drawing  Date 1995-05-17 00:00:00
Prediction:	 [379  56]
GoundTruth:	 ['381' '55']
----------------------------------------
1/1 [==============================] - 0s 34ms/step
Drawing  Date 1995-05-02 00:00:00
Prediction:	 [669  90]
GoundTruth:	 ['692' '88']
----------------------------------------
1/1 [==============================] - 0s 38ms/step
Drawing  Date 1995-04-16 00:00:00
Prediction:	 [584 101]
GoundTruth:	 ['581' '99']
----------------------------------------
1/1 [==============================] - 0s 35ms/step
Drawing  Date 1995-04-01 00:00:00
Prediction:	 [12  3]
GoundTruth:	 ['039' '06']
----------------------------------------
1/1 [==============================] - 0s 44ms/step
Drawing  Date 1995-03-16 00:00:00
Prediction:	 [849  68]
GoundTruth:	 ['847' '67']
----------------------------------------
1/1 [========================

### Prediction with rounding up the results

In [54]:
print('-' * 40)
print('Prediction vs. GoundTruth with rounding up')
for i in range(1,10):
  test = df1.copy()
  test = test.tail((window_length+10-i))
  test = test.head((window_length+1))
  test_Date = df1.iloc[ (test.tail().index[-1]) ]['Date']
  test1 = test.head((window_length))
  test1.drop(['Date'], axis=1, inplace=True)
  test1 = np.array(test1)
  x_test = scaler.transform(test1)
  y_test_pred = model.predict(np.array([x_test]))
  y_test_true = test.drop(['Date'], axis=1, inplace=True)
  y_test_true = test.tail(1)
  print('Drawing  Date', test_Date)
  print('Prediction:\t', scaler.inverse_transform(y_test_pred).astype(int)[0]+1)
  print('GoundTruth:\t', np.array(y_test_true)[0])
  print('-' * 40)

----------------------------------------
Prediction vs. GoundTruth with rounding up
1/1 [==============================] - 0s 59ms/step
Drawing  Date 1995-05-17 00:00:00
Prediction:	 [380  57]
GoundTruth:	 ['381' '55']
----------------------------------------
1/1 [==============================] - 0s 49ms/step
Drawing  Date 1995-05-02 00:00:00
Prediction:	 [670  91]
GoundTruth:	 ['692' '88']
----------------------------------------
1/1 [==============================] - 0s 60ms/step
Drawing  Date 1995-04-16 00:00:00
Prediction:	 [585 102]
GoundTruth:	 ['581' '99']
----------------------------------------
1/1 [==============================] - 0s 75ms/step
Drawing  Date 1995-04-01 00:00:00
Prediction:	 [13  4]
GoundTruth:	 ['039' '06']
----------------------------------------
1/1 [==============================] - 0s 47ms/step
Drawing  Date 1995-03-16 00:00:00
Prediction:	 [850  69]
GoundTruth:	 ['847' '67']
----------------------------------------
1/1 [==============================] -

### Predict the Future Drawing Results on June 1, 2o24

In [57]:
next_Date = '2024-06-01'
print('-' * 40)
print('Predict the Future Drawing on',next_Date)

# Copy and prepare the data for prediction
next = df.copy()
next = next.tail(window_length)
next = np.array(next)
x_next = scaler.transform(next)

# Predict the next values
y_next_pred = model.predict(np.array([x_next]))

# Filter the actual values based on the next date
filtered_df = lt_2[lt_2['Date'] == next_Date]

# Extract the actual values and convert them to a list
actual_values = filtered_df[['top3', 'up2']].values.flatten().astype(int).tolist()

# Calculate the absolute difference
prediction = scaler.inverse_transform(y_next_pred).astype(int)[0].tolist()
diff = [abs(pred - actual) for pred, actual in zip(prediction, actual_values)]

# Print the results
print('Drawing  Date', next_Date)
print('Prediction:\t', prediction)
print('GoundTruth:\t', actual_values)
print('Diff:   \t', diff)
print('-' * 40)

----------------------------------------
Predict the Future Drawing on 2024-06-01
1/1 [==============================] - 0s 112ms/step
Drawing  Date 2024-06-01
Prediction:	 [716, 60]
GoundTruth:	 [593, 42]
Diff:   	 [123, 18]
----------------------------------------


# Evaluate

In [ ]:
evalu = {
    'รอบที่': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'บน': [420, 668, 448, 294, 281, 481, 288, 545, 542, 596],
    'ล่าง': [77, 66, 62, 53, 67, 65, 49, 84, 74, 64],
    'accuracy': [0.9626, 0.9482, 0.9626, 0.964, 0.9698, 0.9712, 0.954, 0.9583, 0.9727, 0.9482]
}

# Convert dictionary to DataFrame
evaluate = pd.DataFrame(evalu)

In [ ]:
df = evaluate.copy()

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(y=evaluate['ล่าง'])
plt.title('Boxplot of ล่าง')
plt.ylabel('ล่าง')
plt.show()

In [ ]:
# Calculate descriptive statistics
min_value = df['ล่าง'].min()
max_value = df['ล่าง'].max()
mean_value = df['ล่าง'].mean()
q1 = df['ล่าง'].quantile(0.25)
q2 = df['ล่าง'].median()
q3 = df['ล่าง'].quantile(0.75)

# Calculate IQR
iqr = q3 - q1

# Determine outliers
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = df['ล่าง'][(df['ล่าง'] < lower_bound) | (df['ล่าง'] > upper_bound)]

# Print the results
print(f'Min value: {min_value}')
print(f'Max value: {max_value}')
print(f'Mean value: {mean_value}')
print(f'Q1 (25th percentile): {q1}')
print(f'Q2 (median, 50th percentile): {q2}')
print(f'Q3 (75th percentile): {q3}')
print(f'Outliers: {outliers.values}')

In [ ]:
# Calculate descriptive statistics for 'บน'
min_value = df['บน'].min()
max_value = df['บน'].max()
mean_value = df['บน'].mean()
q1 = df['บน'].quantile(0.25)
q2 = df['บน'].median()
q3 = df['บน'].quantile(0.75)

# Calculate IQR
iqr = q3 - q1

# Determine outliers for 'บน'
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = df['บน'][(df['บน'] < lower_bound) | (df['บน'] > upper_bound)]

# Print the results for 'บน'
print(f'Min value: {min_value}')
print(f'Max value: {max_value}')
print(f'Mean value: {mean_value}')
print(f'Q1 (25th percentile): {q1}')
print(f'Q2 (median, 50th percentile): {q2}')
print(f'Q3 (75th percentile): {q3}')
print(f'Outliers: {outliers.values}')